#### ML for clustering

##### What is Clustering?
- Clustering is an **unsupervised** machine learning technique with aim to group data that sharing similar features into clusters and so as to discover relationships among data points and features.
- Unlike the supervised learning technique classification where class labels are pre-defined and assigned to data, **clustering is more about discovery than prediction**.
- In the clustering analysis, there is **no outcome variable**, only the features of the dataset.  
![clustering.png](attachment:clustering.png)

#### K-Means Clustering
- Most common clustering algorithm to group data into a set of k clusters, where k is specified by the analyst first.
- It is an iterative algorithm described below:
  - Step 1: Choose k, the number of clusters.
  - Step 2: Choose k random data points known as the initial _cluster centroids_. 
  - Step 3: Compute euclidean distance of each point in the dataset with the cluster centroids, assigning each data point to a cluster centroid that it is closest to. 
  - Step 4: Update the position of the cluster centroid to the mean of the data points in the same cluster.
  - Step 5: Repeat steps 3-4 until the cluster centroids no longer change their positions or maximum number of iterations is reached (Default = 10 for R). 
  
- K-Means picks the best clustering by minimizing the total variation within each cluster. The variation within each cluster is defined by the squared distance of each data point within cluster $i$ and its corresponding centroid, $c_{i}$. The total variation is then the sum of the variation of each data point within each cluster. 
- Note that the k-Means clustering depends on the initial assignment of data points as cluster centroids, so k-means does not always give the same solution on every run.

In [ ]:
# Using k-Means with R base function kmeans()
# Syntax: kmeans(x, centers, iter.max=10, nstart= 1) where the option “x” is a numeric matrix 
# or numeric data frame of the dataset for clustering, “centers” specifies the number of clusters (K) 
# or a set of initial (distinct) cluster centers. If a number, a random set of (distinct) rows in x 
# is chosen as the initial centers. “iter.max” is the maximum number of iterations allowed (default=10)

data(USArrests)
arr <- USArrests
k2 <- kmeans(arr, centers = 2, nstart = 25)
str(k2); k2

'data.frame':	50 obs. of  4 variables:
 $ Murder  : num  13.2 10 8.1 8.8 9 7.9 3.3 5.9 15.4 17.4 ...
 $ Assault : int  236 263 294 190 276 204 110 238 335 211 ...
 $ UrbanPop: int  58 48 80 50 91 78 77 72 80 60 ...
 $ Rape    : num  21.2 44.5 31 19.5 40.6 38.7 11.1 15.8 31.9 25.8 ...


,Murder,Assault,UrbanPop,Rape
,<dbl>,<int>,<int>,<dbl>
Alabama,13.2,236,58,21.2
Alaska,10.0,263,48,44.5
Arizona,8.1,294,80,31.0
Arkansas,8.8,190,50,19.5
California,9.0,276,91,40.6
Colorado,7.9,204,78,38.7


     Murder          Assault         UrbanPop          Rape      
 Min.   : 0.800   Min.   : 45.0   Min.   :32.00   Min.   : 7.30  
 1st Qu.: 4.075   1st Qu.:109.0   1st Qu.:54.50   1st Qu.:15.07  
 Median : 7.250   Median :159.0   Median :66.00   Median :20.10  
 Mean   : 7.788   Mean   :170.8   Mean   :65.54   Mean   :21.23  
 3rd Qu.:11.250   3rd Qu.:249.0   3rd Qu.:77.75   3rd Qu.:26.18  
 Max.   :17.400   Max.   :337.0   Max.   :91.00   Max.   :46.00  

List of 9
 $ cluster     : Named int [1:50] 1 1 1 1 1 1 2 1 1 1 ...
  ..- attr(*, "names")= chr [1:50] "Alabama" "Alaska" "Arizona" "Arkansas" ...
 $ centers     : num [1:2, 1:4] 11.86 4.84 255 109.76 67.62 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:2] "1" "2"
  .. ..$ : chr [1:4] "Murder" "Assault" "UrbanPop" "Rape"
 $ totss       : num 355808
 $ withinss    : num [1:2] 41637 54762
 $ tot.withinss: num 96399
 $ betweenss   : num 259409
 $ size        : int [1:2] 21 29
 $ iter        : int 1
 $ ifault      : int 0
 - attr(*, "class")= chr "kmeans"


K-means clustering with 2 clusters of sizes 21, 29

Cluster means:
     Murder  Assault UrbanPop     Rape
1 11.857143 255.0000 67.61905 28.11429
2  4.841379 109.7586 64.03448 16.24828

Clustering vector:
       Alabama         Alaska        Arizona       Arkansas     California 
             1              1              1              1              1 
      Colorado    Connecticut       Delaware        Florida        Georgia 
             1              2              1              1              1 
        Hawaii          Idaho       Illinois        Indiana           Iowa 
             2              2              1              2              2 
        Kansas       Kentucky      Louisiana          Maine       Maryland 
             2              2              1              2              1 
 Massachusetts       Michigan      Minnesota    Mississippi       Missouri 
             2              1              2              1              2 
       Montana       Nebraska       

- In the output above, 
  - cluster: A vector of integers 1,2 indicating the cluster to which each point is allocated.
  - totss: The total sum of squares.
  - withinss: Vector of within-cluster sum of squares, one component per cluster.
  - tot.withinss: Total within-cluster sum of squares, i.e. sum(withinss).
  - betweenss: The between-cluster sum of squares, i.e. $totss-tot.withinss$.
  - size: The number of points in each cluster.
  
- We see that our grouping results in 2 cluster sizes of 20 and 30, cluster centers (means) for the two groups across the four variables (Murder, Assault, UrbanPop, Rape). We also get the cluster assignment for each observation (i.e. Alabama was assigned to cluster 1, Arkansas was assigned to cluster 2, etc.).

In [ ]:
# To get the optimal K, plot within-cluster variance (using sum of squares) against k
# Recall that K-means minimizes the total within-cluster variance. 
# We use the following algorithm to get the optimal k value:
# 1) Compute k-means clustering for different values of K.
# 2) For each K, calculate the total within-cluster sum of squares (tot.withinss/wss)
# 3) Plot the curve of wss according to the number of clusters K.
# 4) The location of a bend (knee) in the plot is generally considered as an indicator of the appropriate number of clusters.

# Function to compute total within-cluster sum of square 
wcss <- function(k) { 
    kmeans(arr, k, nstart = 10 )$tot.withinss 
} 

# Compute and plot wss for k=1 to k=15 
k.values <- c(1:15)  # vector of k values

# Apply wcss to all k values 
wcss_k <- sapply(k.values, wcss) 
# Plot total within-cluster variance against k
plot(k.values, wcss_k, type="b", pch = 19, frame = FALSE, xlab="Number of clusters K", ylab="Total within-clusters sum of squares")

- From the plot above, the bend occurs at k=4.

In [ ]:
# Final clustering with k=4 
set.seed(100) 
k4.final <- kmeans(arr, 4, nstart = 25); k4.final

K-means clustering with 4 clusters of sizes 10, 14, 10, 16

Cluster means:
     Murder  Assault UrbanPop     Rape
1  2.950000  62.7000 53.90000 11.51000
2  8.214286 173.2857 70.64286 22.84286
3  5.590000 112.4000 65.60000 17.27000
4 11.812500 272.5625 68.31250 28.37500

Clustering vector:
       Alabama         Alaska        Arizona       Arkansas     California 
             4              4              4              2              4 
      Colorado    Connecticut       Delaware        Florida        Georgia 
             2              3              4              4              2 
        Hawaii          Idaho       Illinois        Indiana           Iowa 
             1              3              4              3              1 
        Kansas       Kentucky      Louisiana          Maine       Maryland 
             3              3              4              1              4 
 Massachusetts       Michigan      Minnesota    Mississippi       Missouri 
             2            

#### Hierarchical Clustering
- As an alternative to K-means clustering, hierarchical clustering does not require a particular choice of K.
- A key advantage is that it results in an attractive tree-based representation of the observations, called a **dendrogram**.
- There are many types of hierarchical clustering algorithms, we'll focus on the **agglomerative clustering**.
- Agglomerative clustering works in a bottom-up manner, where starting from each observation, the similarity between every pair of observations is measured using distance measures (e.g. euclidean distance) and points nearest to it are grouped into clusters. At each step of the algorithm, any pair of clusters that are the most similar are combined into a new bigger cluster. This procedure is iterated until all points are members of just one single big cluster (root).
- To measure the similarity between pairs of clusters, we'll use the **complete linkage** cluster agglomeration method.

![agglom.png](attachment:agglom.png)

- Note that the hierarchical clustering is sensitive to scale, so normalization of the numeric variables should be carried out first using R base function scale({dataframe})

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
# Normalization of numeric variables
arr <- scale(arr)

# Dissimilarity matrix 
d <- dist(arr, method = "euclidean") 

# Hierarchical clustering using Complete Linkage 
hc1 <- hclust(d, method = "complete" ) 

# Plot the obtained dendrogram 
plot(hc1, cex = 0.6, hang = -1) 
# Draw a border around the 4 clusters, option “border” specifies colors 
rect.hclust(hc1, k = 4, border = 2:5)

- In the dendrogram above:
 - Each leaf corresponds to one observation. As we move up the tree, observations that are similar to each other are combined into branches, which are themselves fused at a higher height.
 - The height of the fusion, provided on the vertical axis, indicates the (dis)similarity between two observations. The higher the height of the fusion, the less similar the observations are.
 - To determine the optimal number of clusters, we use the elbow graph similar to K-means clustering.